### coinbase-ventures-portfolio

In [1]:
# get category symbols
category = "coinbase-ventures-portfolio"
import requests
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "category": category,
    "order": "market_cap_desc",
    "per_page": 300,
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

# Extract the symbols
defi_symbols = [coin['symbol'].upper() + '/USDT' for coin in data]
print("DeFi Symbols:", defi_symbols)

DeFi Symbols: ['SUI/USDT', 'NEAR/USDT', 'APT/USDT', 'UNI/USDT', 'SEI/USDT', 'IMX/USDT', 'TIA/USDT', 'GRT/USDT', 'WLD/USDT', 'ONDO/USDT', 'AR/USDT', 'LDO/USDT', 'STRK/USDT', 'MATIC/USDT', 'MINA/USDT', 'LUNC/USDT', 'ARKM/USDT', 'SNX/USDT', 'LPT/USDT', 'RSR/USDT', 'CELO/USDT', 'ZRX/USDT', 'BICO/USDT', 'GAL/USDT', 'RPL/USDT', 'WELL/USDT', 'ORCA/USDT', 'AUDIO/USDT', 'GLMR/USDT', 'CFG/USDT', 'INST/USDT', 'SSV/USDT', 'GFI/USDT', 'ZENT/USDT', 'CTK/USDT', 'POND/USDT', 'AUCTION/USDT', 'DODO/USDT', 'MAV/USDT', 'EUL/USDT', 'RSS3/USDT', 'HFT/USDT', 'ACA/USDT', 'ZBCN/USDT', 'PRCL/USDT', 'SLND/USDT', 'CXT/USDT', 'FLIP/USDT', 'TOKE/USDT', 'FORT/USDT', 'DESO/USDT', 'RBN/USDT', 'ROUTE/USDT', 'SD/USDT', 'ECOX/USDT', 'PSTAKE/USDT', '$SMH/USDT', 'RARI/USDT', 'RLY/USDT', 'LMR/USDT', 'TRUF/USDT', 'PTU/USDT', 'BCUT/USDT', 'FLX/USDT', 'EVMOS/USDT', 'NEXT/USDT', 'ASTO/USDT', 'FEI/USDT', 'UX/USDT', 'OPEN/USDT', 'DDX/USDT', 'CQT/USDT', 'TAKI/USDT', 'HXRO/USDT', 'UP/USDT', 'NOTE/USDT', 'SFI/USDT', 'AMKT/USDT', 'VE

In [4]:
# Fetch aviable symbols from Binance
import ccxt

exchange = ccxt.bybit()
exchange.load_markets()
all_symbols = exchange.symbols

# Filter to include only DeFi symbols available on the exchange
available_defi_symbols = [symbol for symbol in defi_symbols if symbol in all_symbols]
print("Available DeFi Symbols:", available_defi_symbols)

Available DeFi Symbols: ['SUI/USDT', 'NEAR/USDT', 'APT/USDT', 'UNI/USDT', 'SEI/USDT', 'IMX/USDT', 'TIA/USDT', 'GRT/USDT', 'WLD/USDT', 'ONDO/USDT', 'AR/USDT', 'LDO/USDT', 'STRK/USDT', 'MINA/USDT', 'LUNC/USDT', 'ARKM/USDT', 'SNX/USDT', 'CELO/USDT', 'ZRX/USDT', 'BICO/USDT', 'RPL/USDT', 'WELL/USDT', 'GLMR/USDT', 'SSV/USDT', 'ZENT/USDT', 'RSS3/USDT', 'HFT/USDT', 'ACA/USDT', 'PRCL/USDT', 'FLIP/USDT', 'FORT/USDT', 'SD/USDT', 'ECOX/USDT', 'PSTAKE/USDT', 'PTU/USDT', 'BCUT/USDT', 'NEXT/USDT', 'FTT/USDT']


In [5]:
# Fetch ohlcv from Bybit
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import plotly.express as px
import plotly.graph_objects as go

symbols = available_defi_symbols
timeframe = '1d'
limit = 252

# Create an empty list to store DataFrames for each symbol
data_frames = []

for symbol in symbols:
    try:
        ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['symbol'] = symbol

        # Convert timestamp to datetime format for readability
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

        data_frames.append(df)
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")

# Concatenate all DataFrames into a single DataFrame
all_data = pd.concat(data_frames, ignore_index=True)
all_data.head()

,timestamp,open,high,low,close,volume,symbol
0,2024-03-10,1.5843,1.6514,1.5150,1.5550,8036897.01,SUI/USDT
1,2024-03-11,1.5550,1.5927,1.4755,1.5721,14040580.82,SUI/USDT
2,2024-03-12,1.5721,1.6353,1.4232,1.5991,24576085.45,SUI/USDT
3,2024-03-13,1.5991,1.6964,1.5733,1.6210,18274402.22,SUI/USDT
4,2024-03-14,1.6210,1.6603,1.4713,1.5674,18876518.39,SUI/USDT


In [6]:
# Check min max date
print(all_data['timestamp'].min(), all_data['timestamp'].max())

2024-03-10 00:00:00 2024-11-16 00:00:00


In [7]:
# Filter dublicates and unnecesary timestamps
# Ensure that 'timestamp' is in datetime format
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])

# Find the latest date in the data
latest_date = all_data['timestamp'].max()

# Calculate the cutoff date for 252 days before the latest date
cutoff_date = latest_date - pd.Timedelta(days=252)

# Filter the data to include only rows within the last 252 days
filtered_data = all_data[all_data['timestamp'] >= cutoff_date]

# Check the new date range to confirm
print("Filtered data range:", filtered_data['timestamp'].min(), "-", filtered_data['timestamp'].max())

# Identify duplicate entries based on 'timestamp' and 'symbol'
duplicates = filtered_data[filtered_data.duplicated(subset=['timestamp', 'symbol'], keep=False)]
print("Duplicate entries:")
duplicates.info()

# Remove duplicate entries, keeping only the first occurrence
filtered_data = filtered_data.drop_duplicates(subset=['timestamp', 'symbol'], keep='first')


Filtered data range: 2024-03-10 00:00:00 - 2024-11-16 00:00:00
Duplicate entries:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  0 non-null      datetime64[ns]
 1   open       0 non-null      float64       
 2   high       0 non-null      float64       
 3   low        0 non-null      float64       
 4   close      0 non-null      float64       
 5   volume     0 non-null      float64       
 6   symbol     0 non-null      object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 0.0+ bytes


In [8]:
# get closed price data
df = filtered_data[['timestamp', 'close', 'symbol']]
df.head()

,timestamp,close,symbol
0,2024-03-10,1.5550,SUI/USDT
1,2024-03-11,1.5721,SUI/USDT
2,2024-03-12,1.5991,SUI/USDT
3,2024-03-13,1.6210,SUI/USDT
4,2024-03-14,1.5674,SUI/USDT


In [9]:
# pivot by timestamp and symbol
df_pivoted = df.pivot(index='timestamp', columns='symbol', values='close')

print(df_pivoted.head())
print(df_pivoted.tail())

symbol      ACA/USDT  APT/USDT  AR/USDT  ARKM/USDT  BCUT/USDT  BICO/USDT  \
timestamp                                                                  
2024-03-10    0.1321   12.7483  41.2125    3.20786    0.30094     0.6163   
2024-03-11    0.1449   13.4642  39.6196    3.06223    0.29308     0.6572   
2024-03-12    0.1883   13.3778  37.9058    3.14473    0.33999     0.6609   
2024-03-13    0.1991   13.1462  37.7186    3.01128    0.44520     0.7119   
2024-03-14    0.1822   14.2954  38.7300    3.21886    0.45555     0.6955   

symbol      CELO/USDT  ECOX/USDT  FLIP/USDT  FORT/USDT  ...  SNX/USDT  \
timestamp                                               ...             
2024-03-10     1.1681     0.2535     8.8851     0.2509  ...    4.3689   
2024-03-11     1.7600     0.2498     8.0133     0.3156  ...    4.7304   
2024-03-12     1.5312     0.2344     7.9010     0.2892  ...    5.0929   
2024-03-13     1.4747     0.2550     7.6185     0.2688  ...    4.9456   
2024-03-14     1.4301     0.2

In [10]:
# Calculate monthly returns
returns_df = df_pivoted.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)
# Convert the index of past_cum_return_df to timezone-aware UTC
returns_df.index = returns_df.index.tz_localize('UTC')
returns_df.tail(7)

symbol,ACA/USDT,APT/USDT,AR/USDT,ARKM/USDT,BCUT/USDT,BICO/USDT,CELO/USDT,ECOX/USDT,FLIP/USDT,FORT/USDT,...,SNX/USDT,SSV/USDT,STRK/USDT,SUI/USDT,TIA/USDT,UNI/USDT,WELL/USDT,WLD/USDT,ZENT/USDT,ZRX/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,-0.035581,0.050551,0.498428,0.359682,-0.123564,0.276261,0.074511,-0.167121,0.005342,0.039916,...,0.072628,0.084160,-0.039967,-0.105438,0.112506,0.414731,0.000000,0.064337,0.083517,0.123163
2024-06-30 00:00:00+00:00,-0.319417,-0.226417,-0.385295,-0.312170,-0.479228,-0.296410,-0.259309,-0.113022,-0.497666,-0.286465,...,-0.297519,-0.083764,-0.413764,-0.198593,-0.411932,-0.066272,0.000000,-0.489087,-0.397609,-0.291458
2024-07-31 00:00:00+00:00,-0.074180,-0.093225,0.045649,-0.293644,-0.235030,-0.278021,-0.118345,-0.171745,0.070565,-0.120612,...,-0.173689,-0.262841,-0.327536,-0.137439,-0.150371,-0.228691,-0.371436,-0.147670,-0.112331,-0.005973
2024-08-31 00:00:00+00:00,-0.107858,0.012428,-0.257002,-0.177279,-0.476288,-0.211278,-0.153487,0.423634,-0.244390,-0.208628,...,-0.190433,-0.358614,-0.232759,0.125689,-0.147958,-0.171003,-0.231142,-0.292019,-0.163178,-0.207594
2024-09-30 00:00:00+00:00,0.119171,0.181101,0.056939,0.413307,0.425511,0.068778,0.420495,-0.241190,0.179322,0.001627,...,0.182506,0.194759,0.233146,1.228711,0.232226,0.243344,-0.376358,0.243642,0.241615,0.153051
2024-10-31 00:00:00+00:00,-0.137346,0.185478,-0.327150,0.105066,-0.288251,-0.066892,-0.039956,-0.154799,-0.053219,-0.238830,...,-0.121596,-0.164856,-0.154897,0.111130,-0.119698,0.030327,-0.304704,0.037445,-0.252747,-0.050523
2024-11-30 00:00:00+00:00,0.114490,0.366378,0.199610,0.302239,0.939060,0.292196,0.085830,-0.013431,0.151401,0.019210,...,0.192718,0.017107,0.258760,0.917533,0.069959,0.184456,-0.052618,0.230029,0.294730,0.202141


In [11]:
# Create a line plot with Plotly
fig = px.line(returns_df, x=returns_df.index, y=returns_df.columns, title="Monthly returns")
# Adjust the layout for a wider plot
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Returns")
# Show the interactive plot
fig.show()

In [12]:
# rolling cumulative for 6 months return
past_cum_return_df = (returns_df + 1).rolling(6).apply(np.prod) - 1

past_cum_return_df.tail(7)

symbol,ACA/USDT,APT/USDT,AR/USDT,ARKM/USDT,BCUT/USDT,BICO/USDT,CELO/USDT,ECOX/USDT,FLIP/USDT,FORT/USDT,...,SNX/USDT,SSV/USDT,STRK/USDT,SUI/USDT,TIA/USDT,UNI/USDT,WELL/USDT,WLD/USDT,ZENT/USDT,ZRX/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-31 00:00:00+00:00,-0.561696,-0.490716,-0.475242,-0.675042,-0.931747,-0.641408,-0.612362,-0.496252,-0.879495,-0.510163,...,-0.694362,-0.485641,-0.853317,-0.487974,-0.720158,-0.582046,-0.516724,-0.843935,-0.515160,-0.637148
2024-09-30 00:00:00+00:00,-0.645708,-0.550847,-0.410061,-0.436293,-0.894638,-0.645558,-0.483415,-0.525930,-0.766080,-0.468939,...,-0.661290,-0.585999,-0.798531,-0.073029,-0.613707,-0.428977,-0.698609,-0.774509,-0.398015,-0.560100
2024-10-31 00:00:00+00:00,-0.476592,0.044644,-0.491086,-0.151173,-0.814475,-0.490051,-0.189951,-0.441337,-0.543859,-0.606303,...,-0.476426,-0.531378,-0.697390,0.723815,-0.486261,0.082037,-0.790444,-0.576634,-0.550165,-0.313742
2024-11-30 00:00:00+00:00,-0.395146,0.358696,-0.592574,-0.187034,-0.589538,-0.483684,-0.181419,-0.338247,-0.477589,-0.614141,...,-0.417807,-0.560361,-0.603229,2.695074,-0.505909,-0.094086,-0.801470,-0.510726,-0.462478,-0.265487


In [13]:
# Show cumulative return from past 6 months
fig = px.line(past_cum_return_df, x=past_cum_return_df.index, y=past_cum_return_df.columns, title="Rolling cumulative for 6 months return")
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Cumulative Returns")
fig.show()

In [14]:
# Define formation and measurement period
# Define the dates with timezone information
end_of_measurement_period = dt.datetime(2024, 9, 30, tzinfo=pytz.UTC)
formation_period = dt.datetime(2024, 10, 31, tzinfo=pytz.UTC)

end_of_measurement_period_return_df = past_cum_return_df.loc[end_of_measurement_period]
# Transpose the DataFrame so that tickers become rows and returns become a single column
end_of_measurement_period_return_df = end_of_measurement_period_return_df.T
end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()
# Rename columns for clarity
end_of_measurement_period_return_df.columns = ['Symbol', 'Return']
end_of_measurement_period_return_df.head()

,Symbol,Return
0,ACA/USDT,-0.645708
1,APT/USDT,-0.550847
2,AR/USDT,-0.410061
3,ARKM/USDT,-0.436293
4,BCUT/USDT,-0.894638


In [15]:
# highest momentum in the positive direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()]

Symbol    FTT/USDT
Return     -0.0418
Name: 10, dtype: object

In [16]:
# highest momentum in the negative direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()]

Symbol    BCUT/USDT
Return    -0.894638
Name: 4, dtype: object

In [19]:
# rank symbols with quantiles
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 11, labels=False)
end_of_measurement_period_return_df.head(7)

,Symbol,Return,rank
0,ACA/USDT,-0.645708,2
1,APT/USDT,-0.550847,5
2,AR/USDT,-0.410061,9
3,ARKM/USDT,-0.436293,8
4,BCUT/USDT,-0.894638,0
5,BICO/USDT,-0.645558,2
6,CELO/USDT,-0.483415,7


In [20]:
# Create a bar plot to visualize ranks
fig = px.bar(
    end_of_measurement_period_return_df, 
    x='Symbol', 
    y='Return', 
    color='rank',  # Color by rank to visually distinguish groups
    title="Asset Returns and Ranks",
    labels={'rank': 'Rank'},
)

# Customize layout for readability
fig.update_layout(
    xaxis_title="Symbol",
    yaxis_title="Return",
    coloraxis_colorbar=dict(title="Rank"),
    width=1000,
    height=600
)

fig.show()


In [31]:
# select highest rank
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==10,"Symbol"].values
long_stocks

array(['FTT/USDT', 'NEAR/USDT', 'ONDO/USDT', 'SUI/USDT'], dtype=object)

In [22]:
# select lowest rank
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,"Symbol"].values
short_stocks

array(['BCUT/USDT', 'FLIP/USDT', 'STRK/USDT', 'WLD/USDT'], dtype=object)

In [23]:
# long strategy return
from dateutil.relativedelta import relativedelta

long_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(long_stocks)]
long_return_df

symbol
FTT/USDT     0.148473
NEAR/USDT    0.499864
ONDO/USDT    0.418580
SUI/USDT     0.917533
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [24]:
# short strategy return
short_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(short_stocks)]
short_return_df

symbol
BCUT/USDT    0.939060
FLIP/USDT    0.151401
STRK/USDT    0.258760
WLD/USDT     0.230029
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [26]:
# Define Strategy tester class
class strategyTester:
    """Calculate metrics based on cumulative return"""

    def __init__(self, df: pd.Series, period: str, riskfree_rate: float = 0.03, initial_wealth: int = 1000):
        self.annualized_return_value = self.annualized_return(df, period)
        self.annualized_volatility_value = self.annualized_volatility(df, period)
        self.sharpe_ratio_value = self.sharpe_ratio(df, riskfree_rate)
        self.max_drawdown_value = self.max_drawdown(df, initial_wealth)
        
    # annualized return
    @staticmethod
    def annualized_return(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        annualized_return = (1 + df) ** (factor / len(df)) - 1
        print(f"Annualized return: {annualized_return}")
        return annualized_return

    # annualized volatility
    @staticmethod
    def annualized_volatility(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        volatility = df.std() * (factor ** 0.5)
        print(f"Annualized volatility: {volatility}")
        return volatility

    # Sharpe ratio
    @staticmethod
    def sharpe_ratio(df: pd.Series, riskfree_rate: float = 0.03):
        excess_return = df.mean() - riskfree_rate
        volatility = df.std()
        sharpe_ratio = excess_return / volatility if volatility != 0 else np.nan
        print(f"Sharpe ratio: {sharpe_ratio}")
        return sharpe_ratio

    # Drawdown calculation
    @staticmethod
    def drawdown(return_series: pd.Series, initial_wealth: float = 100):
        prior_peaks_series = return_series.cummax()
        drawdown_series = (return_series - prior_peaks_series) / prior_peaks_series
        return pd.DataFrame({
            "Wealth Index": return_series,
            "Prior Peaks": prior_peaks_series,
            "Drawdown": drawdown_series
        })
    
    # Max drawdown
    @staticmethod
    def max_drawdown(df: pd.Series, initial_wealth):
        drawdown_df = strategyTester.drawdown(df, initial_wealth)
        max_drawdown = drawdown_df["Drawdown"].min()
        print(f"Max drawdown: {max_drawdown} with initial wealth: {initial_wealth}")
        return max_drawdown


In [32]:
# long momentum strategy test
long_momentum_strategy_test = strategyTester(long_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
FTT/USDT     0.514824
NEAR/USDT    2.374084
ONDO/USDT    1.854710
SUI/USDT     6.050641
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 1.103583115977283
Sharpe ratio: 1.4631083966296063
Max drawdown: -0.16261185876955972 with initial wealth: 1000


In [33]:
# short momentum strategy test
short_momentum_strategy_test = strategyTester(short_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
BCUT/USDT    6.290770
FLIP/USDT    0.526441
STRK/USDT    0.994476
WLD/USDT     0.860999
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 1.2666771146475782
Sharpe ratio: 0.9976870982387566
Max drawdown: -0.8387737014998548 with initial wealth: 1000


In [30]:
# Comparison of monthly Long and Short momentum strategies
# Initialize an empty figure
fig = go.Figure()

# Add the first strategy (e.g., long strategy)
fig.add_trace(go.Bar(x=long_return_df.index, y=long_return_df.values, marker_color='blue', name="Long"))

# Add the second strategy (e.g., short strategy)
fig.add_trace(go.Bar(x=short_return_df.index, y=short_return_df.values, marker_color='red', name="Short"))

# Update layout for title and axis labels
fig.update_layout(
    title="Comparison of monthly Long and Short momentum strategies",
    width=1200,
    height=400,
    xaxis_title="Symbol",
    yaxis_title="Cumulative Returns",
    legend_title="Strategies",
)
# Show the plot
fig.show()

In [34]:
# Annualized Return and Max Drawdown from Long and Short Strategies
# Initialize an empty figure
fig = go.Figure()

# Add traces for Long strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Long)", "Max Drawdown (Long)"], 
    y=[long_momentum_strategy_test.annualized_return_value.mean(), long_momentum_strategy_test.max_drawdown_value], 
    marker_color='blue', 
    name="Long"
))

# Add traces for Short strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Short)", "Max Drawdown (Short)"], 
    y=[short_momentum_strategy_test.annualized_return_value.mean(), short_momentum_strategy_test.max_drawdown_value], 
    marker_color='red', 
    name="Short"
))

# Update layout
fig.update_layout(
    title="Annualized Return and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode='group',  # Group bars by category
    width=800,
    height=400
)

# Show the plot
fig.show()



In [35]:
# Annualized Volatility and Max Drawdown from Long and Short Strategies
# Create a figure
fig = go.Figure()

# Add bars for the Long Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(long)", "Sharpe Ratio(long)"], 
    y=[long_momentum_strategy_test.annualized_volatility_value, long_momentum_strategy_test.sharpe_ratio_value],
    name="Long",
    marker_color='blue'
))

# Add bars for the Short Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(short)", "Sharpe Ratio(short)"], 
    y=[short_momentum_strategy_test.annualized_volatility_value, short_momentum_strategy_test.sharpe_ratio_value],
    name="Short",
    marker_color='red'
))

# Update layout for grouped bars
fig.update_layout(
    title="Annualized Volatility and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Metric Value",
    barmode='group',  # Group bars side-by-side
    width=800,
    height=400
)

# Show the plot
fig.show()
